In [2]:
import geopandas as gpd

In [4]:
# Open soil map
soil_pedo = gpd.read_file(
    "../data/BRO_Soil_Map_Netherlands/BRO_Soil_Map_Netherlands.gpkg",
    layer="areaofpedologicalinterest",
)

soil_area = gpd.read_file(
    "../data/BRO_Soil_Map_Netherlands/BRO_Soil_Map_Netherlands.gpkg",
    layer="soilarea",
)

soil_profile = gpd.read_file(
    "../data/BRO_Soil_Map_Netherlands/BRO_Soil_Map_Netherlands.gpkg",
    layer="soilarea_normalsoilprofile",
)

soil_profile_info = gpd.read_file(
    "../data/BRO_Soil_Map_Netherlands/BRO_Soil_Map_Netherlands.gpkg",
    layer="normalsoilprofiles",
)

In [5]:
soil_pedo_label = soil_pedo["pedologicalinterest"].unique()
print(";".join(soil_pedo_label))

soil_pedo_label_en = "Heavily raised terrain;Terp;Heavily leveled;Heavily processed terrain;Heavily excavated terrain;Quarry;Water;Swamp;Built-up area;Dike;Mine dump;Upland;Land raised with household waste".split(
    ";"
)

soil_pedo_label_remap = {}
for x in range(len(soil_pedo_label_en)):
    soil_pedo_label_remap[soil_pedo_label[x]] = soil_pedo_label_en[x]

soil_pedo["label_en"] = soil_pedo["pedologicalinterest"]
soil_pedo = soil_pedo.replace({"label_en": soil_pedo_label_remap})
soil_pedo

Sterk opgehoogd terrein;Terp;Sterk ge-egaliseerd;Sterk verwerkt terrein;Sterk afgegraven terrein;Groeve;Water;Moeras;Bebouwd gebied;Dijk;Mijnstort;Bovenland;Met huisvuil opgehoogde gronden


,maparea_id,maparea_collection,beginlifespan,endlifespan,pedologicalinterest,geometry,label_en
0,V2023-1.areaofpedologicalinterrest.0000000027,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((59361 387558, 59330 387662, 59300 38...",Heavily raised terrain
1,V2023-1.areaofpedologicalinterrest.0000000028,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((59017 388003, 58891 388008, 58875 38...",Heavily raised terrain
2,V2023-1.areaofpedologicalinterrest.0000000022,Actualisatie met uitgebreide veldverkenning 2010,None,None,Sterk opgehoogd terrein,"POLYGON ((256321.176 522624.758, 256306.835 52...",Heavily raised terrain
3,V2023-1.areaofpedologicalinterrest.0000000032,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((118393 383729, 118427 383738, 118347...",Heavily raised terrain
4,V2023-1.areaofpedologicalinterrest.0000000033,Eerste opname met uitgebreide veldverkenning,None,None,Sterk opgehoogd terrein,"POLYGON ((108288 396779, 108445 396814, 108464...",Heavily raised terrain
...,...,...,...,...,...,...,...
5877,V2023-1.areaofpedologicalinterrest.0000052287,Actualisatie met uitgebreide veldverkenning 2021,None,None,Sterk opgehoogd terrein,"POLYGON ((159841.528 467325.127, 159855.795 46...",Heavily raised terrain
5878,V2023-1.areaofpedologicalinterrest.0000052619,Actualisatie met digitale bodemkartering 2022,None,None,Sterk opgehoogd terrein,"POLYGON ((135333.542 433538.717, 135329.464 43...",Heavily raised terrain
5879,V2023-1.areaofpedologicalinterrest.0000051998,Revisie met detailbodemkaart 2020,None,None,Sterk opgehoogd terrein,"POLYGON ((147113.441 476279.781, 147111.103 47...",Heavily raised terrain
5880,V2023-1.areaofpedologicalinterrest.0000052520,Actualisatie met digitale bodemkartering 2022,None,None,Bebouwd gebied,"POLYGON ((128222 438343.891, 128316.687 438499...",Built-up area


In [9]:
# Inspire id
soil_profile_info_inspire_id = soil_profile_info["inspireid"]
soil_profile_id = list(
    map(lambda x: int(x.split(".")[-1]), soil_profile_info_inspire_id)
)
soil_profile_info["normalsoilprofile_id"] = soil_profile_id
soil_profile_info

,soilunit,region,validfrom,beginlifespanversion,inspireid,lowervalue,uppervalue,othersoilname,soilprofile_id,normalsoilprofile_id
0,hVb,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.1010,0.0,1.2,Koopveengronden op bosveen (of eutroof broekveen),1010,1010
1,ohVb,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.1020,0.0,1.2,Koopveengronden op bosveen (of eutroof broekveen),1020,1020
2,hVs,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.1030,0.0,1.2,Koopveengronden op veenmosveen,1030,1030
3,hVsc,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.1040,0.0,1.2,Koopveengronden op veenmosveen,1040,1040
4,hVc,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.1050,0.0,1.2,"Koopveengronden op zeggeveen, rietzeggeveen of...",1050,1050
...,...,...,...,...,...,...,...,...,...,...
363,Rd10A,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.90216230,0.0,1.2,Kalkhoudende ooivaaggronden; lichte zavel,90216230,90216230
364,Rd90A,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.90216240,0.0,1.2,Kalkhoudende ooivaaggronden; zware zavel en li...,90216240,90216240
365,Rd10C,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.90216250,0.0,1.2,Kalkloze ooivaaggronden; lichte zavel,90216250,90216250
366,Rd90C,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.90216260,0.0,1.2,Kalkloze ooivaaggronden; zware zavel en lichte...,90216260,90216260


In [15]:
# Soil area
soil_area_profile = soil_area.merge(
    soil_profile, left_on="maparea_id", right_on="maparea_id"
)
soil_area_profile = soil_area_profile.merge(
    soil_profile_info, left_on="normalsoilprofile_id", right_on="normalsoilprofile_id"
)
soil_area_profile

,maparea_id,maparea_collection,beginlifespan,endlifespan,soilslope,geometry,normalsoilprofile_id,soilunit,region,validfrom,beginlifespanversion,inspireid,lowervalue,uppervalue,othersoilname,soilprofile_id
0,V2023-1.soilarea.0000007891,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((250817.437 487812.5, 250841.063 4878...",9028040,bEZ23,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.9028040,0.0,1.2,Hoge bruine enkeerdgronden; lemig fijn zand,9028040
1,V2023-1.soilarea.0000007024,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206726.516 536030.313, 206716.062 53...",1100,aVz,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.1100,0.0,1.2,"Madeveengronden op zand zonder humuspodzol, be...",1100
2,V2023-1.soilarea.0000007025,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((206429.234 535641.25, 206418.125 535...",1100,aVz,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.1100,0.0,1.2,"Madeveengronden op zand zonder humuspodzol, be...",1100
3,V2023-1.soilarea.0000007026,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((207894.395 536426.077, 207881.125 53...",1100,aVz,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.1100,0.0,1.2,"Madeveengronden op zand zonder humuspodzol, be...",1100
4,V2023-1.soilarea.0000007800,Eerste opname met uitgebreide veldverkenning,None,None,Niet opgenomen,"POLYGON ((260372.859 484005.406, 260353.734 48...",9028040,bEZ23,None,2000-01-01,2000-01-01,sgm.V2023-1.profile.9028040,0.0,1.2,Hoge bruine enkeerdgronden; lemig fijn zand,9028040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46920,V2023-1.soilarea.0000050836,Actualisatie met uitgebreide veldverkenning 2010,None,None,Niet opgenomen,"POLYGON ((263628.525 518695.221, 263617.186 51...",2120,zWz,Nederland zonder Flevoland,2000-01-01,2000-01-01,sgm.V2023-1.profile.2120,0.0,1.2,Moerige eerdgronden met een zanddek en een moe...,2120
46921,V2023-1.soilarea.0000050837,Actualisatie met digitale bodemkartering 2013,None,None,Niet opgenomen,"POLYGON ((195713 497640, 195799 497739, 195880...",2120,zWz,Nederland zonder Flevoland,2000-01-01,2000-01-01,sgm.V2023-1.profile.2120,0.0,1.2,Moerige eerdgronden met een zanddek en een moe...,2120
46922,V2023-1.soilarea.0000051241,Actualisatie met digitale bodemkartering 2018,None,None,Niet opgenomen,"POLYGON ((165128 478373, 165106 478559, 165093...",4045,kHn21,Flevoland,2000-01-01,2000-01-01,sgm.V2023-1.profile.4045,0.0,1.2,Veldpodzolgronden; leemarm en zwak lemig fijn ...,4045
46923,V2023-1.soilarea.0000051242,Actualisatie met digitale bodemkartering 2018,None,None,Niet opgenomen,"POLYGON ((181298.203 507226.375, 181222.266 50...",1225,kVc,Flevoland,2000-01-01,2000-01-01,sgm.V2023-1.profile.1225,0.0,1.2,"Waardveengronden op zeggeveen, rietzeggeveen o...",1225


In [36]:
soil_names = soil_area_profile["othersoilname"].unique()
print("|".join(soil_names))

soil_names_en = "High brown enkeerd soils; loamy fine sand|Madeveen soils on sand without humus podzol, starting shallower than 1.2 m|Laarpodzol soils; loamy and weak loamy fine sand|Laarpodzol soils; loamy fine sand|Kamppodzol soils; loamy and weak loamy fine sand|Veldpodzol soils; loamy and weak loamy fine sand|Beekeerd soils; loamy and weak loamy fine sand|Bergbrik soils; silty loam|Radebrik soils; sandy loam|Radebrik soils; silty loam|Calcareous polder vague soils; light loam, profile gradient 5|Calcareous polder vague soils; heavy loam, profile gradient 5|Gooreerd soils; loamy and weak loamy fine sand|Calcareous polder vague soils; light loam, profile progression 5|Calcareous polder vague soils; heavy clay, profile progression 5|Low-calcareous polder vague soils; loam, profile progression 3, or 3 and 4, or 4|Low-calcareous polder vague soils; clay, profile progression 5|Low-calcareous polder vague soils; heavy loam, profile progression 5|Knippige polder vague soils; loam, profile progression 3|Low-calcareous polder vague soils; clay, profile progression 3, or 3 and 4, or 4|Non-calcareous polder vague soils (brown basin soil); heavy clay, profile progression 3, or 3 and 4, or 4|Calcareous polder vague soils; light clay, profile progression 5|Radebrik soils; fine-sandy light loam|Holtpodzol soils; loamy fine sand|Peat soils with a peat topsoil or peat interlayer on matured loam or clay|High brown peat soils; coarse sand|Elder peat soils on peat moss peat|Dune vague soils; loam-poor and weak loamy fine sand|Song soils; clay, profile progression 1|High brown peat soils; loam-poor and weak loamy fine sand|Cut-up polder vague soils; light loam, profile progression 5|Cut-up polder vague soils; heavy loam, profile progression 5|Calcareous polder vague soils; heavy loam and light clay, profile progression 5|Made peat soils on sedge peat, reed sedge peat or marsh peat|Made peat soils on sand with humus podzol, starting shallower than 1.2 m|Calcareous polder vague soils; light loam, profile progression 2|Calcareous flat vague soils; very fine sand|Flat vague soils; coarse sand|Elder peat soils on sand with humus podzol, starting shallower than 1.2 m|Boggy earth soils with a boggy topsoil on sand|High black enkeerd soils; loam-poor and weak loamy fine sand|Calcareous polder vague soils; light loam, profile progression 5|Calcareous drecht vague soils; loam, profile progression 1|Eo vague soils with rust starting deeper than 0.8 m; silty loam; colluvium in slope foot or leaching fan|Beekeerd soils; loamy fine sand|Elder peat soils on sedge peat, reed sedge peat or (mesotrophic) marsh peat|Moorish moor soils with a loam or clay cover and a boggy interlayer on sand|Non-calcareous polder vague soils; heavy loam and light clay, profile progression 5|Calcareous leek/wood soils; loam, profile progression 5|Low-calcareous drechtvaag soils; loam and light clay, profile progression 1|Calcareous leek/wood soils; clay, profile progression 5|Low-calcareous polder vague soils; loam and light clay, profile progression 2|Calcareous flat vague soils; moderately fine sand|Lake peat soils on sand without humus podzol, starting shallower than 1.2 m|Low-calcareous drechtvaag soils; heavy clay, profile progression 1|Low-calcareous polder vague soils; light loam, profile progression 5|Eye-waste soils; heavy loam and clay|Tochteerd soils; clay|Holtpodzol soils; coarse sand|Eye-waste soils with rust starting deeper than 0.8 m; sandy loam in situ|Moorish eerd soils with a boggy topsoil or boggy intermediate layer on unripened loam or clay|Elder peat soils on sand without humus podzol, starting shallower than 1.2 m|Eye-waste soils, light loam|Haarpodzol soils; loam-poor and weakly loamy fine sand|Cut-up polder-waste soils; clay, profile progression 5|Cut-up polder-waste soils; clay, profile progression 3|Cut-up polder-waste soils; loam, profile progression 2|Dune-waste soils; coarse sand|Holtpodzol soils; loamy and weak loamy fine sand|Gooreerd soils; loamy fine sand|Cut-up polder vague soils; clay, profile progression 4, or 4 and 3|High black enkeerd soils; coarse sand|Cut-up polder vague soils; loam, profile progression 4, or 4 and 3|Beekeerd soils; coarse sand|Gooreerd soils; coarse sand|Frost vague soils; coarse sand|Veldpodzol soils; loamy fine sand|High black enkeerd soils; loamy fine sand|Non-calcareous polder vague soils; heavy loam and light clay, profile progression 4|Calcareous oovaag soils; light loam|Flat vague soils; loamy and weak loamy fine sand|Flat vague soils; loamy fine sand|Oovaag soils with rust starting deeper than 0.8 m; silty loam in situ|Field podzol soils; coarse sand|Haar podzol soils; coarse sand|Moist eerd soils with a sand cover and a moory interlayer on sand|Loo podzol soils; coarse sand|Waardveen soils on sand, starting shallower than 1.2 m|Meadow peat soils on sand, starting shallower than 1.2 m|Leek/woudeerd soils; loam, profile progression 5, or 5 and 2, or 2|Non-calcareous drechtvaag soils; profile progression 1|Leek/woudeerd soils; sandy loam; colluvium in valley|Loo podzol soils; loam-poor and weak loamy fine sand|Loo podzol soils; loamy fine sand|Polder vaag soils; light loam|Meadow peat soils on sedge peat, reed sedge peat or (mesotrophic) marsh peat|Polder vague soils; heavy loam|Meadow peat soils on peat moss peat|Boggy podzol soils with a boggy topsoil|Leek/woudeerd soils; clay, profile progression 3, or 3 and 4, or 4|Polder vague soils; clay|Low enkeerd soils; loam-poor and weak loamy fine sand|Non-calcareous polder vague soils; heavy clay, profile progression 3, or 3 and 4|Gorsvaag soils; heavy loam and clay; no sand starting shallower than 0.8 m|Sludge vague soils; no sand starting shallower than 0.8 m|Caustic nesvaag soils; heavy loam|Caustic polder vague soils; clay, profile progression 2|Low-calcium polder vague soils; loam, profile progression 2|Low-calcium polder vague soils; loam and light clay, profile progression 3, or 3 and 4, or 4|Calicy drecht vague soils; clay, profile progression 1|Garden soils; light loam, profile progression 5, or 5 and 2, or 2|Calicy nes vague soils; clay|Peat soils on sedge peat, reed sedge peat or (mesotrophic) marsh peat|Peat soils on (usually unripened) loam or clay, starting shallower than 1.2 m|Peat soils on sand, starting shallower than 1.2 m|Peat soils with a peat colonial cover on sand without humus podzol, starting shallower than 1.2 m|Calcareous polder vague soils; heavy loam, profile progression 2|Low-calcareous nesvaag soils; clay|Calcareous enkeerd soils; moderately fine sand|Borsvaag soils; heavy loam and clay; sand starting shallower than 0.8 m|Waardveen soils on sedge peat, reed sedge peat or (mesotrophic) marsh peat|Garden soils; sandy loam|Calcareous polder vague soils; heavy clay, profile progression 3, or 3 and 4, or 4|Tochteerd soils; loam|Waardveen soils on peat moss peat|Calyx-poor polder vague soils; clay, profile progression 2|Moorish eerd soils with a peat colony cover and a boggy intermediate layer on sand|Elder peat soils on (usually unripened) loam or clay, starting shallower than 1.2 m|Purified peat soils on forest peat (or eutrophic marsh peat)|Knippolder vague soils; heavy clay, profile progression 4, or 4 and 3|Knippolder vague soils; loam and light clay, profile progression 3|Knippolder vague soils; heavy clay, profile progression 3|Kicky soil|Calyx-containing flat vague soils; weak and strong loam, clayey, extremely fine sand|Peat podzol soils with a loam or clay cover and a peat interlayer|Knippolder vague soils; loam and light clay, profile progression 4, or 4 and 3|Value peat soils on forest peat (or eutrophic bog peat)|Value peat soils on (usually unripened) loam or clay, starting shallower than 1.2 m|Peat podzol soils with a peat colonial cover and a peat interlayer|Peat soils with a peat colonial cover on sand with humus podzol, starting shallower than 1.2 m|Peat soils with a peat colonial cover on sedge peat, reed sedge peat or marsh forest peat|Peat peat soils on peat moss peat|Calcareous dune vague soils; fine sand|Non-calcareous floodplain soils; light loam|Calcareous floodplain soils; heavy loam and light clay|Calcareous leek/woodland soils; clay, profile progression 3, or 3 and 4 or 4|Mud-flooded soils; sand starting shallower than 0.8 m|Calcareous leek/woodland soils; loam, profile progression 5|Polder-flooded soils; sandy loam in situ|Very shallow boulder clay, pot clay, etc.|Other clay soils|Calcareous flat-flooded soils; extremely fine sand|Calcareous leek/woodland soils; clay, profile progression 5|Calcareous brook-flooded soils; very fine and moderately fine sand|Liedeerd soils; loam, profile progression 1|Frost-flooded soils; loamy fine sand|Pasture peat soils on forest peat (or eutrophic bog peat)|Calcareous leek/forest soils; loam, profile progression 2|Non-calcareous polder vague soils; heavy clay, profile progression 4|Non-calcareous polder vague soils; loam and light clay, profile progression 3, or 3 and 4|Non-calcareous floodplain soils; heavy loam and light clay|Calcareous flat vague soils; coarse sand|Lake peat soils on sand with humus podzol, starting shallower than 1.2 m|Lake peat soils on sedge peat, reed sedge peat or bog peat|Boggy podzol soils with a humus-containing sand cover and a boggy intermediate layer|Calcareous dune vague soils; fine sand, partially decalcified|Frost vague soils; loam-poor and weak loamy fine sand|Calcareous flat vague soils; coarse sand, partially decalcified|Peat degradation area"
soil_names_en = soil_names_en.split("|")
soil_names_en

remap_soil = {}
for x in range(len(soil_names)):
  remap_soil[soil_names[x]] = soil_names_en[x]
remap_soil

Hoge bruine enkeerdgronden; lemig fijn zand|Madeveengronden op zand zonder humuspodzol, beginnend ondieper dan 1.2 m|Laarpodzolgronden; leemarm en zwak lemig fijn zand|Laarpodzolgronden; lemig fijn zand|Kamppodzolgronden; leemarm en zwak lemig fijn zand|Veldpodzolgronden; leemarm en zwak lemig fijn zand|Beekeerdgronden; leemarm en zwak lemig fijn zand|Bergbrikgronden; siltige leem|Radebrikgronden; zandige leem|Radebrikgronden; siltige leem|Kalkrijke poldervaaggronden; lichte zavel, profielverloop 5|Kalkrijke poldervaaggronden; zware zavel, profielverloop 5|Gooreerdgronden; leemarm en zwak lemig fijn zand|Kalkarme poldervaaggronden; lichte zavel, profielverloop 5|Kalkrijke poldervaaggronden; zware klei, profielverloop 5|Kalkarme poldervaaggronden; zavel, profielverloop 3, of 3 en 4, of 4|Kalkarme poldervaaggronden; klei, profielverloop 5|Kalkarme poldervaaggronden; zware zavel, profielverloop 5|Knippige poldervaaggronden; zavel, profielverloop 3|Kalkarme poldervaaggronden; klei, profiel

IndexError: list index out of range